# BIBLIOTECA DE FUNCIONES <a class="anchor" id="0-0"></a>

    Este notebook es un documento auxiliar que sirve de repositorio para describir y/o consultar las funciones que se van creando a lo largo del proyecto. 

1. [Función: Genera EDA](#1)

2. [Función: Convierte un archivo PKL a JSON](#2)

3. [Función: Convierte PKL a DF](#3)

4. [Función: Convierte consulta Mongo DB a DF](#4)

5. [Función: Convierte de milisegundos a fecha larga](#5)

6. [Función: Convierte JSON a DF](#6)

7. [Función: Convierte milisegundos a fecha corta](#7)

8. [Función: Crea ID](#8)

9. [Función: Tokeniza texto](#9)

10. [Función: Analiza columna](#10)

11. [Función: Crea respaldo de la base de datos (dump)](#11)

12. [Función: Inyecta registros en Mongo por lotes](#12)

13. [Función: Limpia una base de datos (delete)](#13)

14. [Función: Lee JSON a DF e inyecta DF a JSON](#14)

15. [Función: Crea colecciones en BBDD Mongo DB](#15)

16. [Función: Crea colecciones en BBDD Mongo DB (2)](#16)

17. [Función: Mostrar tablas en SQLite](#17)

18. [Procedimiento: Genera imágenes WORDCLOUD (Alt 1)](#18)

19. [Procedimiento: Genera imágenes WORDCLOUD (Alt 2)](#19)

20. [Función: Convierte consulta a Mongo en un df (Google, Paso 1 de 3)](#20)

21. [Función: Función: Divide dirección (Google, paso 2 de 3)](#21)

22. [Función: Divide ZIP (Google,paso 3 de 3)](#22)

23. [Función: Convierte datos de categorías a un diccionario](#23)

24. [Función: Agrega Estado](#24)

25. [Función: Elimina datos de colección Mongo DB](#25)

26. [Función: Analiza faltantes](#26)

27. [Función: Rango de fechas](#27)

28. [Función: ](#28)

29. [Función: ](#29)

30. [Función: ](#30)

## 1. Función: Genera EDA<a class="anchor" id="1"></a>

    Útil si el df es tabular, no está pensado para datos anidados.

In [ ]:
# Función que permite generar un informe tipo EDA del dataframe.
# **************************************************************
# Esta función se usará para el diagnóstico inicial y final de los datos.
# Genera información valiosa para conocer el estadoy consistencia de los datos. 
#
def f_reportedf(dataframe, nombre):
    """
    Función que permite generar un informe tipo EDA del dataframe.
    Recibe como parámetros, un dataframe y una cadena de texto 
    que describe el set de datos. 
    Retorna:
    
        a) Nombre del DF, número de registros y columnas
        
        b) Si el dataset tiene duplicados y cuántos.
        
        b) 2 registros de muestra, tomados al azar.
        
        c) Nombres de las columnas
        
        d) Estadísticas básicas de las columnas numéricas
        
        e) Valores únicos por cada columna
        
        f) Porcentaje de nulos por cada columna
        
        g) Estadísticas básicas de las columnas categóricas
        
        h) Tabla descriptiva del set de datos
    """
    print(f"RESUMEN ANÁLISIS EXPLORATORIO DE DATOS: {nombre.upper()}")
    print("-"*50)
    print(f"El DF tiene {dataframe.shape[0]} registros y {dataframe.shape[1]} columnas")
    print("-"*50)
    if dataframe.duplicated().sum() == 0: 
        print('El dataset no tiene duplicados')
    else:
        print(f"El DF tiene {dataframe.duplicated().sum()} registros duplicados") 
    print("-"*50)
    print("Para chequear estructura y tipos de datos, se muestran dos (2) filas seleccionadas al azar:")
    display(dataframe.sample(2).style.background_gradient(cmap='Oranges_r').set_properties(**{'font-family': 'Segoe UI'}).hide_index())
    print("-"*50)
    print(f"El dataframe {nombre} tiene las siguientes columnas y tipos de datos:")
    for i, col in enumerate(dataframe.columns, start=1):
        print(i,') ',col, type(dataframe[col].iloc[0]))
    print("-"*50)
    print("Las estadísticas básicas de las variables numéricas son:")
    display(dataframe.select_dtypes(exclude='object').describe().T.style.background_gradient(cmap='Oranges_r').set_properties(**{'font-family': 'Segoe UI'}))
    print("-"*50)
    print("El numero de valores distintos de cada columna es:")
    for col in dataframe.columns:
        if len(dataframe[col].value_counts()) > 15:
            print(col, len(dataframe[col].value_counts()))
        else:
            print(col, len(dataframe[col].value_counts()))
            print(f"Los valores son: {dataframe[col].unique()}")
    print("-"*50)
    print("El porcentaje de nulos por columna:")
    for i, col in enumerate(dataframe.isnull().sum()):
        print(f"{dataframe.isnull().sum().index[i]}: {col/dataframe.shape[0]*100}")
    print("-"*50)
    print("Las estadísticas básicas de las variables categóricas son:")
    try:
        if dataframe.select_dtypes(include='object').shape[1] > 0:
            display(dataframe.select_dtypes(include='object').describe().T.style.background_gradient(cmap='Oranges_r').set_properties(**{'font-family': 'Segoe UI'}))
        else:
            print("No se encontraron variables categóricas en el dataframe.")
    except AttributeError:
        print("No se encontraron variables categóricas en el dataframe.")
    print("-"*50)
    print("Tabla descriptiva que incluye todas las variables:")
    display(dataframe.describe(include='all').style \
  .format(precision=2, thousands=",", decimal=".").background_gradient(cmap='Oranges_r').set_properties(**{'font-family': 'Segoe UI'}))

## 2. Función: Convierte un archivo PKL a JSON<a class="anchor" id="2"></a>

    Necesario para convertir uno de los archivos del set de datos originales.

    Recibe una ruta, que lleva a archivo .pkl, y retorna un archivo en formato json en el mismo directorio de origen u otra ruta que se especifique. 

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
import json
import pickle

# Función: Convierte de PKL a JSON (f_pkl2json())
# ***********************************************
#
def f_pkl2json(v_entra, v_sale):
    """
    Recibe un archivo en formato PKL 
    y lo convierte a formato JSON
    """
    # Dentro de un ciclo try/except para manejar excepciones
    try:
        # Lee el .pkl
        with open(v_entra, 'rb') as v_file:
            data = pickle.load(v_file)

        # Convierte a JSON
        with open(v_sale, 'w') as v_jsonFile:
            json.dump(data, v_jsonFile, indent=4)

        print(f"Conversión finalizada. El archivo JSON se ha guardado en: {v_sale}") # Se guarda enel directorio activo o ruta especificada
    except Exception as e:
        print(f"Error al convertir el archivo: {e}") # Muestra el error, en caso de falla

# Ejecución
# *********
#
v_entra = 'D://business.pkl'         # Reemplazar 'nom_archivo.pkl' y la ruta
v_sale  = 'D://test040823_biz.json'  # Reemplazar 'nom_archivo.json' con el nombre del archivo destino
f_pkl2json(v_entra, v_sale)          # Obs: verificar que el pkl sea serializable (que no sea, por ejemplo, un modelo de ML)

## 3. Función: Convierte PKL a DF<a class="anchor" id="3"></a>

Lee archivo local, en formato PKL, y lo convierte a DF

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
import pandas as pd

# Función: Convierte de PKL a DF (f_pkl2df())
# ***********************************************
# 
def f_pkl2df(v_pkl):
    """
    Recibe un archivo en formato PKL
    lo covierte a dataframe de Pandas. 
    """
    
    try:
        # Lee el .pkl y lo convierte en un df
        df = pd.read_pickle(v_pkl)

        print("Conversión terminada, 1 df ha sido creado.")
        return df
    except Exception as e:
        print(f"Error al convertir el archivo: {e}")
        return None

# Ejecución
# *********
#
v_pkl = 'D://business.pkl'       # Reemplazar 'nom_archivo.pkl' y la ruta

df_ybusiness = f_pkl2df(v_pkl)
if df_ybusiness is not None:
    print(df_ybusiness.head(3))  # Muestrea 3 registros

## 4. Función: Convierte consulta Mongo DB a DF<a class="anchor" id="4"></a>

    Convierte una consulta a la BBDD, recibe como parámetros nombre de la BBDD y colección  

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
import pandas as pd
from pymongo import MongoClient

# Función: Convierte consulta en Mongo DB a dataframe
# ***************************************************
#

def f_deMongoaDF(v_nomDB, v_nomCollection):
    '''
    Función que hace una consulta en Mongo DB
    Recibe  : El nombre de una BBDD y una colección
    Retorna : Un df 
    '''
    
    # String de conexión
    client     = MongoClient('localhost', 27017)
    db         = client[v_nomDB]
    collection = db[v_nomCollection]

    # Pipe para consulta a la colección
    query = collection.find({}, {
        'business_id' : 1,  # El valor 0 indica no se incluye y 1 si se incluye
        'name'        : 1,
        'address'     : 1,
        'postal_code' : 1,
        '_id'         : 0   # Índice interno de Mongo DB, por eso se excluye
    })

    # Con el resultado de la consulta, crea un DF
    df_ybiz = pd.DataFrame(query)       # 'df_' por 'dataframe', 'y' por 'Yelp', y 'biz' por 'business'

    # Cerrar la conexión con la base de datos
    client.close()

    return df_ybiz

# Ejecución
# *********
#
df_ybiz = f_deMongoaDF('yelpdb', 'business')

## 5. Función: Convierte de milisegundos a fecha larga<a class="anchor" id="5"></a>

    Función usada para convertir el campo time de la tabla de rviews en la bbdd Google Maps.

    Args    : Un valor en milisegundos
    Retorna : Una fcha en formato largo

[Volver - Tabla de Contenidos](#0-0)

In [29]:
import datetime
# Función: Convierte fecha
# ************************
# Recibe  : Un entero de la forma "1597168272670"
# Retorna : Una fecha de la forma "Fecha convertida: 2020-08-11 13:51:12.670000"
# Nota: Para correr proceso masivo se debe eliminar el mensaje y sólo
# retornar el valor. Se usa en combinación con una función lambda
# para recorrer una variable o dentro de otras funciones.
# Usar en las reviews de GOOGLE

def f_convierteFecms(v_claveval):
    # Como el set de datos no es claro, se verifica si el valor 
    # es en milisegundos o segundos y se ajusta si es necesario
    if v_claveval > 9999999999:   # Si el valor es mayor a 10 dígitos, 
        v_claveval /= 1000        # se asume que está en milisegundos

    return datetime.datetime.fromtimestamp(v_claveval)

# Testeo
# ******
#
v_dataEjemplo   = 1597168272670
v_fecConvertida = f_convierteFecms(v_dataEjemplo)
print("Fecha convertida:", v_fecConvertida)

Fecha convertida: 2020-08-11 13:51:12.670000


## 6. Función: Convierte JSON a DF<a class="anchor" id="6"></a>

    Convierte un archivo JSON a dataframe.
    
Args: 

v_ruta  = Una ruta al archivo  
v_nomdf = Nombre para el df (string)
  
Retorna:
Un dataframe con el nombre ingresado como parámetro
 
Nota: Se requiere esta función para poder automatizar la carga masiva de los 611 archivos de google.

[Volver - Tabla de Contenidos](#0-0)

In [3]:
# Librerías mínimas
# *****************
import pandas as pd
import json

# Función que convierte un archivo JSON a dataframe de Pandas
# ***********************************************************
# Args: 
# v_ruta  = Una ruta al archivo  
# v_nomdf = Nombre para el df (string)
#  
# Retorna:
# Un dataframe
# 
# Nota: Se requiere esta función para poder automatizar la carga masiva
# de los 611 archivos de google.
#

def f_json2DF(v_ruta, v_nomdf):
    """
    Convierte un archivo JSON a dataframe de Pandas
    
    Args: 
    v_ruta  = Una ruta al archivo  
    v_nomdf = Nombre para el df (string)
    
    Retorna:
    Un dataframe
    """
    try:
        data = []
        with open(v_ruta, "r") as archivo:
            for linea in archivo:
                try:
                    objeto_json = json.loads(linea)
                    data.append(objeto_json)
                except json.JSONDecodeError:
                    print(f"Error al leer el archivo en la línea: {linea.strip()}")

        v_tmpdf = pd.DataFrame(data)  # Almacena temporalmente el df, para poder asignarle el nombre al final.
        globals()[v_nomdf] = v_tmpdf  # Usamos globals() para crear una variable global con el nombre especificado
        return v_tmpdf
    except Exception as e:
        print(f"Error al leer el archivo JSON: {str(e)}")
        return None

# Testing
# *******
#
# Parámetros
v_nomdf = "df_georgia1"
v_ruta  = "C:\\Descargas\\GogRev\\review-Georgia\\1.json"  # Sugerencia: Usar barras dobles en la ruta

v_dfresul = f_json2DF(v_ruta, v_nomdf)

if v_dfresul is not None:
    print(f'Archivo JSON procesado con éxito, se ha creado el df "{v_nomdf}"')
else:
    print("Error al procesar el archivo JSON.")

Archivo JSON procesado con éxito, se ha creado el df "df_georgia1"


In [38]:
df_georgia1.head()

,user_id,name,time,rating,text,pics,resp,gmap_id,fecha,id_gmap,id_guser,tokenizado
0,104227684932200174741,Brent Powell,1599604812269,5,Great hometown folk. Rob takes good care of al...,None,None,88f16e41928ff687:0x883dad4fd048e8f8,2020-09-08,Sq5Kv,e3Gri,great hometown folk rob take good care us choo...
1,105572260232544349578,Evan Thomas,1502078169146,5,Very friendly hometown pharmarcy. Always helpf...,None,None,88f16e41928ff687:0x883dad4fd048e8f8,2017-08-06,tdfgP,EgScN,friendli hometown pharmarci alway help welcom ...
2,103955436073218468595,Caleb Howell,1502571430852,5,None,None,None,88f16e41928ff687:0x883dad4fd048e8f8,2017-08-12,Um0VE,FGvag,None
3,106974620002238817356,ashlee crawford,1494161075405,5,None,None,None,88f16e41928ff687:0x883dad4fd048e8f8,2017-05-07,Q1q7h,K75E9,None
4,104845669459415055814,T Paulk,1495139229501,5,None,None,None,88f16e41928ff687:0x883dad4fd048e8f8,2017-05-18,OKTuH,pk5Qj,None


In [36]:
print(type(df_georgia1['time'][0]))

<class 'numpy.int64'>


In [35]:
df_georgia1.shape

(150000, 12)

## 7. Función: Convierte milisegundos a fecha corta<a class="anchor" id="7"></a>

    Convierte un número en milisegundos a formato de fecha corta (AAAA-mm-dd).
    
Args: 

v_nomdf  = Un df 

v_nomcol = Nombre de una columna (string)
  
Retorna:
Un dataframe con una nueva columna que contiene fechas equivalentes en formato corto
 
Nota: Se requiere esta función para poder transformar los 611 (55) archivos de google.

[Volver - Tabla de Contenidos](#0-0)

In [22]:
# Función: Convierte a fecha corta
# ********************************
# No ejecutar

def f_addFecCorta(v_nomdf, v_nomcol):
    """
    Agrega una nueva columna 'fecha' al DataFrame 'v_nomdf' utilizando la columna 'v_nomcol'.
    Convierte los valores de milisegundos en formato de fecha corta (AAAA-mm-dd).
    
    Recibe:
    v_nomdf   = DataFrame
    v_nomcol  = Nombre de la columna a utilizar
    
    Retorna:
    El DataFrame 'v_nomdf' con la nueva columna 'fecha' agregada en formato de fecha corta
    """
    v_nomdf['fecha'] = v_nomdf[v_nomcol].apply(lambda x: None if x is None else datetime.datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d'))
    return v_nomdf

# Testing
# *******

df_test = f_addFecCorta(df_georgia1, 'time')
print(df_test)


                      user_id             name           time  rating  \
0       104227684932200174741     Brent Powell  1599604812269       5   
1       105572260232544349578      Evan Thomas  1502078169146       5   
2       103955436073218468595     Caleb Howell  1502571430852       5   
3       106974620002238817356  ashlee crawford  1494161075405       5   
4       104845669459415055814          T Paulk  1495139229501       5   
...                       ...              ...            ...     ...   
149995  103688335448019423220     Coleman Cann  1469806243836       5   
149996  108667197296689482669    Samantha Hsia  1476031120937       5   
149997  104019065991738839483   Amber Duarrani  1522705429533       5   
149998  102539365088090137632       Tal Peretz  1452646765537       5   
149999  106041280627830987707   Tracie Griffin  1538997600841       5   

                                                     text  \
0       Great hometown folk. Rob takes good care of al...   
1

## 8. Función: Crea ID<a class="anchor" id="8"></a>

    Crea id único.
    
Args: 

v_nomdf    = Un df
v_nuevacol = Nombre de una nueva columna a insertar (string) 
v_largoKey = Parámetro, setea el número de caracteres para formar el 'id'
  
Retorna:
Una nueva columna en un df que contiene un id corto (más liviano) para reemplazar el id original.
 
Nota: Función de transformación.

[Volver - Tabla de Contenidos](#0-0)

In [26]:
import pandas as pd
import random
import string

def f_creaId(v_nomdf, v_nuevacol, v_largoKey=5):
    # Verifica si la columna 'v_nuevacol' ya existe en el df
    if v_nuevacol not in v_nomdf.columns:
        v_nomdf[v_nuevacol] = None
    
    # Crea la nueva columna 'v_nuevacol' con el nuevo 'id'
    def f_idUnico():
        v_chars = string.ascii_letters + string.digits
        v_new_id = ''.join(random.choice(v_chars) for _ in range(v_largoKey))
        while v_new_id in v_nomdf[v_nuevacol].values:
            v_new_id = ''.join(random.choice(v_chars) for _ in range(v_largoKey))
        return v_new_id

    # Aplica la función f_idUnico a la columna v_nuevacol
    v_nomdf[v_nuevacol] = v_nomdf.apply(lambda row: f_idUnico(), axis=1)

    return v_nomdf

# Testeo
# ******
# Se repite para user_id y para gmap_id
# id_guser, incluye una 'g' por google
# eventualmente, los id de yelp, llevarían
# una 'y', ejemplo, 'id_yuser'

v_nomdf    = df_test
v_nuevacol = "id_guser"
v_largoKey = 5

df_test = f_creaId(v_nomdf, v_nuevacol, v_largoKey)
print(df_test.head(5))

                 user_id             name           time  rating  \
0  104227684932200174741     Brent Powell  1599604812269       5   
1  105572260232544349578      Evan Thomas  1502078169146       5   
2  103955436073218468595     Caleb Howell  1502571430852       5   
3  106974620002238817356  ashlee crawford  1494161075405       5   
4  104845669459415055814          T Paulk  1495139229501       5   

                                                text  pics  resp  \
0  Great hometown folk. Rob takes good care of al...  None  None   
1  Very friendly hometown pharmarcy. Always helpf...  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                               gmap_id       fecha id_gmap id_guser  
0  88f16e41928ff687:0x883dad4fd048e8f8  2020-09-08   Sq5Kv    e3Gri  
1  88f16e41928ff687:0x883dad4fd048e8f8  20

## 9. Función: Tokeniza texto<a class="anchor" id="9"></a>

    Pre-procesa el campo 'texto', aplicando proceso de tokenización.
    
Args: 

v_nomdf   = Un df.

v_nomcol = Nombre de la columna que contiene los mensajes
  
Retorna:

Una nueva columna llamada 'tokenizado' en el df que contiene un el texto pre-procesado
Nota: Disminuye en 40% el peso de la columna (más liviano).
 
Nota: Función de transformación orientada a mejorar la performance y reduce un proceso completo dentro de Azure.

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
# Librerías para tokenización
# ***************************
#
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import sys

nltk.download('punkt')
nltk.download('stopwords')

In [31]:
# Función: Tokeniza campo de mensaje
# **********************************
# Convierte un campo de mensaje y lo pre-procesa
# dejándolo como tokens, listos para ser procesados 
# con NLP 
# 
def f_tokeniza(v_nomdf, v_nomcol):
    # Verifica si la columna 'tokenizado' ya existe en el df
    if 'tokenizado' not in v_nomdf.columns:
        v_nomdf['tokenizado'] = None
    
    # Pre-procesamiento de mensajes: Tokeniza, elimina stopwords y stemming
    def f_preprocesaMsj(text):
        if text is None:
            return None
        
        # Tokenización
        tokens = word_tokenize(text)
        
        # Elimina stopwords y signos de puntuación
        tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
        
        # Stemming
        stemmer = PorterStemmer()
        tokens  = [stemmer.stem(word) for word in tokens]
        
        return ' '.join(tokens)
    
    # Aplica la función f_preprocesaMsj a la columna v_nomcol
    v_nomdf['tokenizado'] = v_nomdf[v_nomcol].apply(f_preprocesaMsj)

    # Calcula el peso en bytes de las columnas v_nomcol y 'tokenizado'
    size_v_nomcol   = v_nomdf[v_nomcol].astype(str).apply(sys.getsizeof).sum()
    size_tokenizado = v_nomdf['tokenizado'].astype(str).apply(sys.getsizeof).sum()

    # Calcula % de disminución en Mb
    porc_menos = ((size_v_nomcol - size_tokenizado) / (1024 * 1024)) / (size_v_nomcol / (1024 * 1024)) * 100
    
    return v_nomdf, porc_menos

# Testeo
# ******
#data = {
#    'text': [
#        "Great hometown folk. Rob takes good care of al",
#        "Very friendly hometown pharmarcy. Always help",
#        None,
#        None,
#        None
#    ]
#}
#df_test = pd.DataFrame(data)

v_nomdf  = df_georgia1
v_nomcol = "text"

df_test, porc_menos = f_tokeniza(v_nomdf, v_nomcol)
print(df_test.head(5))
print(f"La columna original pesa   : {size_v_nomcol}")
print(f"La columna tokenizada pesa : {size_tokenizado}")
print(f"Reducción porcentual     : {porc_menos:.2f}%")


                 user_id             name           time  rating  \
0  104227684932200174741     Brent Powell  1599604812269       5   
1  105572260232544349578      Evan Thomas  1502078169146       5   
2  103955436073218468595     Caleb Howell  1502571430852       5   
3  106974620002238817356  ashlee crawford  1494161075405       5   
4  104845669459415055814          T Paulk  1495139229501       5   

                                                text  pics  resp  \
0  Great hometown folk. Rob takes good care of al...  None  None   
1  Very friendly hometown pharmarcy. Always helpf...  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                               gmap_id       fecha id_gmap id_guser  \
0  88f16e41928ff687:0x883dad4fd048e8f8  2020-09-08   Sq5Kv    e3Gri   
1  88f16e41928ff687:0x883dad4fd048e8f8  

## 10. Función: Analiza columna<a class="anchor" id="10"></a>

    Función informativa, herramienta auxuliar.

Args: 

v_nomdf   = Un df.

v_nomcol = Nombre de una columna a analizar
  
Retorna:

Nulos, no nulos y porcentaje.

Permite saber si la columna es util o debe eliminarse
 

[Volver - Tabla de Contenidos](#0-0)

In [34]:
# Función: Analiza columna
# ************************
# Args: 
# v_nomdf  = Un dataframe 
# v_nomcol = Nombre de una columna
# Retorna:
# Un diccionario con nulos, no nulos y porcentaje sobre el total
#
def f_analizaCol(v_nomdf, v_nomcol):
    """
    Analiza una columna en un df, calcula el número de registros no nulos, 
    registros nulos, y el porcentaje de registros nulos sobre el total de 
    registros.
    
    Args:
    v_nomdf   = DataFrame
    v_nomcol  = Nombre de la columna a analizar
    
    Retorna:
    Un diccionario con la siguiente información:
    {
        'no_nulos'        : Número de registros no nulos,
        'nulos'           : Número de registros nulos,
        'porcentaje_nulos': Porcentaje de registros nulos sobre total
    }
    """
    total_registros  = v_nomdf.shape[0]
    no_nulos         = v_nomdf[v_nomcol].count()
    nulos            = total_registros - no_nulos
    porcentaje_nulos = (nulos / total_registros) * 100 if no_nulos > 0 else 0

    return {
        'no_nulos': no_nulos,
        'nulos': nulos,
        'porcentaje_nulos': porcentaje_nulos,
    }

# Testing
# *******
#
v_nomdf  = df_georgia1
v_nomcol = 'resp'

resultados = f_analizaCol(v_nomdf, v_nomcol)
print("Número de registros no nulos:", resultados['no_nulos'])
print("Número de registros nulos:", resultados['nulos'])
print("Porcentaje de registros nulos sobre total:", resultados['porcentaje_nulos'], "%")

Número de registros no nulos: 29077
Número de registros nulos: 120923
Porcentaje de registros nulos sobre total: 80.61533333333334 %


## 11. Función: Crea respaldo de la base de datos (dump)<a class="anchor" id="11"></a>

    Función de mantenimiento y transporte.

Args: 

v_nomdb    = Nombre de la base de datos en Mongo

v_nomcolec = Nombre de la colección

v_ruta     = Ruta donde guardar el respaldo
  
Retorna:

Un archivo con el respaldo de la BBDD para ser restaurado en la misma locación o montarlo en otro servidor

NOTA: Requiere instalar herramientas adicionales de Mongo para crear el respaldo (dump)


[Volver - Tabla de Contenidos](#0-0)

In [32]:
from pymongo import MongoClient
import subprocess

def f_dumpearBase(v_nomdb, v_nomcolec, v_ruta):
    """
    Realiza un dump de una colección en una base de datos MongoDB y lo almacena en un archivo.
    
    Parámetros:
    v_nomdb    = Nombre de la base de datos MongoDB
    v_nomcolec = Nombre de la colección en la base de datos
    v_ruta     = Ruta del archivo de dump
    
    Retorna:
    Mensaje indicando si el proceso fue exitoso o fallido
    
    Requiere : Tner instaladas utiliddes de Mongo para respaldo (dump)
    """
    try:
        # Conexión a MongoDB
        client = MongoClient("localhost", 27017)
        db     = client[v_nomdb]
        
        # Ejecuta dump
        comando = ["mongodump", "--db", v_nomdb, "--collection", v_nomcolec, "--out", v_ruta]
        subprocess.run(comando)
        
        # Cerrar la conexión a la base de datos
        client.close()
        
        return "Dump exitoso"
    except Exception as e:
        return f"Error al realizar el dump: {str(e)}"

# Ejemplo de uso
v_nomdb    = "googlebd"
v_nomcolec = "grevAlaska"
v_ruta     = "C:\\Descargas\\GogRev\\review-Alaska"
resultado  = f_dumpearBase(v_nomdb, v_nomcolec, v_ruta)
print(resultado)

Error al realizar el dump: [WinError 2] El sistema no puede encontrar el archivo especificado


## 12. Función: Inyecta registros en Mongo por lotes<a class="anchor" id="12"></a>

    Convierte un dataframe en lotes para ser insertados en MongoDB.
    
Args:

v_df       = DataFrame a insertar

v_nomdb    = Nombre de la base de datos en MongoDB

v_nomcolec = Nombre de la colección en MongoDB

v_tamLote  = Tamaño de cada lote (por defecto 75000)

Retorna:

Un mensaje confirmando que los registros han sido insertados con éxito.


NOTA: Mongo limita las cargas masivas a 100K registros por vez, se controla este parámetro, cargando por bloques menores a 100K.


[Volver - Tabla de Contenidos](#0-0)

In [39]:
# Librerías mínimas
# *****************
#
import pandas as pd
from pymongo import MongoClient

# Función: Inyecta registros en Mongo por lotes menores a 100K registros
# **********************************************************************
#

def f_inyectaMongo(v_df, v_nomdb, v_nomcolec, v_tamLote=75000):
    """
    Convierte un dataframe en lotes para ser insertados en MongoDB.
    
    Recibe: 
    v_df       = DataFrame a insertar
    v_nomdb    = Nombre de la base de datos en MongoDB
    v_nomcolec = Nombre de la colección en MongoDB
    v_tamLote  = Tamaño de cada lote (por defecto 75000)
    """
    try:
        # Establecer conexión con MongoDB
        cliente = MongoClient("localhost", 27017) 
        db = cliente[v_nomdb]
        coleccion = db[v_nomcolec]
        
        # Calcular cantidad de lotes completos y remanente
        total_registros = v_df.shape[0]
        cantidad_lotes = total_registros // v_tamLote
        remanente_registros = total_registros % v_tamLote
        
        # Dividir el DataFrame en bloques completos de tamaño v_tamLote
        bloques = [v_df[i * v_tamLote : (i + 1) * v_tamLote] for i in range(cantidad_lotes)]
        
        # Insertar cada bloque en la base de datos
        for bloque in bloques:
            documentos = bloque.to_dict(orient='records')
            coleccion.insert_many(documentos)
        
        # Insertar el último bloque o remanente de registros
        if remanente_registros > 0:
            ultimo_bloque = v_df[cantidad_lotes * v_tamLote :]
            documentos_restantes = ultimo_bloque.to_dict(orient='records')
            coleccion.insert_many(documentos_restantes)
        
        # Cerrar la conexión a la base de datos
        cliente.close()
        
        return True, total_registros

    except Exception as e:
        return False, str(e)

# Ejecución
# *********
# Parámetros
v_ruta     = "C:\\Descargas\\GogRev\\review-Georgia\\1.json"  # Se deben usar dobles barras en la ruta
v_nomdf    = df_georgia1
v_nomdb    = "googledb"
v_nomcolec = "grevGeorgia"

# Suponiendo que ya se ha cargado 'df_georgia1' desde un archivo JSON
v_dfresul = f_inyectaMongo(v_nomdf, v_nomdb, v_nomcolec)

if v_dfresul[0]:
    print(f"Carga exitosa. Se insertaron {v_dfresul[1]} documentos en la BBDD '{v_nomdb}', colección '{v_nomcolec}'.")
else:
    print(f"Error al intentar cargar {v_dfresul[1]} documentos en la BBDD '{v_nomdb}', colección '{v_nomcolec}'.")

Carga exitosa. Se insertaron 150000 documentos en la BBDD 'googledb', colección 'grevGeorgia'.


## 13. Función: Limpia una base de datos (delete)<a class="anchor" id="12"></a>

    Elimina todos los registros insertados en un BBDD MongoDB.
    
Args:

v_nomdb    = Nombre de la BBDD

v_nomcolec = Nombre de la colección

Retorna:

Un mensaje confirmando que los registros han sido eliminados con éxito.


[Volver - Tabla de Contenidos](#0-0)

In [40]:
from pymongo import MongoClient

def f_resetColec(v_nomdb, v_nomcolec):
    """
    Borra todos los registros de una colección en una base de datos de MongoDB.
    
    Args: 
    v_nomdb    = Nombre de la base de datos en MongoDB
    v_nomcolec = Nombre de la colección en MongoDB
    
    Retorna:
    Mensaje "Se han borrado {número de documentos borrados}""
    """
    try:
        # Conexión con MongoDB
        cliente = MongoClient("localhost", 27017) 
        db      = cliente[v_nomdb]
        colec   = db[v_nomcolec]
        
        # Borra TODOS los documentos de la colección
        v_resul = colec.delete_many({})
        
        # Cerrar la conexión a la base de datos
        cliente.close()
        
        return f"Se han borrado {v_resul.deleted_count} documentos"
    except Exception as e:
        return f"Error al borrar documentos: {str(e)}"

# Testeo
# *********

v_nomdb    = "googledb"
v_nomcolec = "grevGeorgia"

mensaje = f_resetColec(v_nomdb, v_nomcolec)
print(mensaje)

Se han borrado 150000 documentos


## 14. Función: Lee JSON a DF e inyecta DF a JSON<a class="anchor" id="14"></a>

    Convierte un archivo JSON a dataframe, luego inyecta los registros a la BBDD MONGO.
    
OBS: Aunque suene redundante, no se halló un procedimiento más eficiente para hacer carga masiva. Se intentó cargar lotes de reviews, pero Mongo tiene varias restricciones, la principal, es que los lotes de registros deben ser de menos de 100K, y cada archivo individual tiene 150K, entonces, obliga a leer el archivo en forma individual y cargarlo en lotes de tamaño archivo/2 o, 75K, debiendo controlar este parámetro. 

Nota: El procedimiento se puede combinar con una lista de df o una rutina que cargue los directorios con un ciclo for.

Args:  
v_nomdf    = Nombre de un dataframe

v_nomdb    = Nombre una BBDD Monggo DB

v_nomcolec = Nombre de la colección 

v_tamLote  = Un parámetro numérico que va de 1 a 99999, se fija en 75000.

Retorna:

Un mensaje confirmando la carga exitosa 

[Volver - Tabla de Contenidos](#0-0)

In [17]:
import pandas as pd
from pymongo import MongoClient

def f_inyectaMongo(v_nomdf, v_nomdb, v_nomcole, v_tamLote=75000):
    """
    Convierte un archivo JSON de 150K registros en 2 lotes de 75K, 
    para manejar la restricción de Max=99999 registros de Mongo DB. 
    
    Recibe: 
    v_nomdf    = Nombre de un dataframe
    v_nomdb    = Nombre una BBDD Monggo DB
    v_nomcolec = Nombre de la colección 
    v_tamLote  = Un parámetro numérico que va de 1 a 99999, se fija en 75000.
    """
    
    try:
        # Establece conexión con MongoDB
        v_cliente = MongoClient("localhost", 27017) 
        v_db      = v_cliente[v_nomdb]
        v_colec   = v_db[v_nomcolec]

        # Lógica para calcular los lotes
        v_numRegs   = v_nomdf.shape[0]       # Nro. de registros en el df.
        v_qttyLotes = v_numRegs // v_tamLote # Calcula nro. de bloques completos
        v_restoRegs = v_numRegs % v_tamLote  # Remanente de registros
        
        # Inicializar la lista de bloques
        lst_bloques = []

        # Dividir el DataFrame en bloques completos de tamaño v_tamLote
        for i in range(v_qttyLotes):
            bloque = df[i * v_tamLote : (i + 1) * v_tamLote]
            lst_bloques.append(bloque)
        
        # Inserta cada bloque en la base de datos
        for v_block in v_numBloques[:-1]:
            v_docs = v_block.to_dict(orient='records')
            v_coleccion.insert_many(v_docs)

        # Insertar el último bloque o remanente de registros
        v_lastBlock     = v_numBloques[-1]
        v_docsRestantes = v_lastBlock.to_dict(orient = 'records')
        coleccion.insert_many(v_docsRestantes)

        # Agregar el último bloque con el remanente de registros
        if v_restoRegs > 0:
            v_lastBlock = df[v_qttyLotes * v_tamLote :]
            bloques.append(v_lastBlock)

        # Cerrar la conexión a la base de datos
        cliente.close()

        return True, v_totalDocs

    except Exception as e:
        return False, str(e)

# Testing
# *******
#

# Parámetros
v_ruta     = "C:\\Descargas\\GogRev\\review-Georgia\\1.json"  # Se deben usar dobles barras en la ruta
v_nomdf    = "df_georgia1"
v_nomdb    = "googledb"
v_nomcolec = "grevGeorgia"

# Suponiendo que ya se ha cargado 'df_arizona1' desde un archivo JSON
v_dfresul = f_inyectaMongo(df_georgia1, 'googledb', 'grevGeorgia')

if v_dfresul[0]:
    print(f"Carga exitosa. Se insertaron {v_dfresul[1]} documentos en la BBDD {v_nomdb}.")
else:
    print(f"Error al intentar inyectar {v_dfresul[1]} documentos en la BBDD {v_nomdb}")

Carga exitosa. Se insertaron 150000 documentos en la base de datos.


## 15. Función: Crea colecciones en BBDD Mongo DB<a class="anchor" id="15"></a>

    Función: Crea los objetos "Colección" en la BBDD de Mongo DB con los estados de USA


Args: 
Una lista con los nombres de Estados de USA (51)

Retorna : 

Un mensaje confirmando la creación de las 51 colecciones (folders) en la BBDD MongoDB (localhost:27017)
Si la carpeta ya existe, la omite.
Se requiere para automatizar proceso masivo
Trabaja con la función f_multiproceso()
Agrega un prefijo "grev" por "Google Reviews", para identificar la fuente + nombre del Estado

Nota: 

El procedimiento se puede combinar con una lista de df o una rutina que cargue los directorios con un ciclo for.

Como la lista es un parámetro, permite hacer el modelo escalableo ajustable a las necesidades, para 'n' Estados

[Volver - Tabla de Contenidos](#0-0)

In [29]:
lst_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "DistrictOfColumbia","Florida", "Georgia", "Hawaii", "Idaho", "Illinois",
    "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "NewHampshire",
    "NewJersey", "NewMexico", "NewYork", "NorthCarolina", "NorthDakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "RhodeIsland", "SouthCarolina", "SouthDakota", "Tennessee",
    "Texas", "Utah", "Vermont", "Virginia", "Washington", "WestVirginia", "Wisconsin", "Wyoming"
            ]
len (lst_states)

51

In [ ]:
# Librerías minimas
# *****************
#
import pandas as pd
from pymongo import MongoClient

# Función: Crea los objetos "Colección" en la BBDD de Mongo DB con los estados de USA
# ***********************************************************************************
# Recibe  : Una lista con los nombres de Estados de USA (51)
# Retorna : La creación de las colecciones en la BBDD MongoDB, localhost:27017
#           Un mensaje confirmando la creación de cada carpeta.
#           Si ya existe, la omite.
# Se requiere para automatizar proceso masivo
# Trabaja con la función f_multiproceso()
# Agrega un prefijo "grev" por "Google Reviews", para identificar la fuente + nombre del Estado

def f_crearColecciones(lst_states, db_nombre):
    """
    Crea las colecciones en la BBDD Mongo DB
    Recibe  :   Una lista con los nombres de Estados de USA (51)
    Retorna :   La creación de las colecciones en la BBDD MongoDB, localhost:27017
                Un mensaje confirmando la creación de cada carpeta.
                Si ya existe, la omite.
    """
    lst_colecciones = []
    try:
        # Conexión a la base de datos MongoDB
        cliente = MongoClient("localhost", 27017)
        db      = cliente[db_nombre]

        # Itera sobre la lista de Estados y crear cada colección
        for v_state in lst_states:
            # Agregar el prefijo "grev" al nombre del estado
            v_nomColec = f"grev{v_state.capitalize()}"

            # Verificar si la colección ya existe en la base de datos
            if v_nomColec in db.list_collection_names():
                print(f"Colección '{v_nomColec}' ya existe, next!")
            else:
                # Crear la colección con el nombre creado "v_nomColec"
                db.create_collection(v_nomColec)
                lst_colecciones.append(v_nomColec)
                print(f"Colección '{v_nomColec}' fue creada con éxito en la bbdd.")

        # Cerrar la conexión a la base de datos
        cliente.close()

        return True, lst_colecciones

    except Exception as e:
        return False, str(e)

# Testing
# *******
#

# Nombre de la base de datos
v_nomdb = "googledb"

# Llamada a la función para crear las colecciones
resultado, lst_colecciones = f_crearColecciones(lst_states, v_nomdb)

if resultado:
    print(f"Todas las colecciones fueron creadas con éxito en la base de datos {v_nomdb}: {lst_colecciones}")
else:
    print(f"Error al crear las colecciones: {lst_colecciones}")

## 16. Función: Crea colecciones en BBDD Mongo DB (2)<a class="anchor" id="16"></a>

    Función: Crea los objetos "Colección" en la BBDD de Mongo DB con los estados de USA

Args: 
Una lista con los nombres de Estados de USA (51)

Retorna : 

Un mensaje confirmando la creación de las 51 colecciones (folders) en la BBDD MongoDB (localhost:27017)
Si la carpeta ya existe, la omite.
Se requiere para automatizar proceso masivo
Trabaja con la función f_multiproceso()
Agrega un prefijo "grev" por "Google Reviews", para identificar la fuente + nombre del Estado

Nota: 

El procedimiento se puede combinar con una lista de df o una rutina que cargue los directorios con un ciclo for.

Como la lista es un parámetro, permite hacer el modelo escalableo ajustable a las necesidades, para 'n' Estados

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
from pymongo import MongoClient

# Función: Crea los objetos "Colección" en la BBDD de Mongo DB con los estados de USA
# ***********************************************************************************
# Recibe  : Una lista con los nombres de Estados de USA (51)
# Retorna : La creación de las colecciones en la BBDD MongoDB, localhost:27017
#           Un mensaje confirmando la creación de cada carpeta.
#           Si ya existe, la omite.
# Se requiere para automatizar proceso masivo
# Trabaja con la función f_multiproceso()
# Agrega un prefijo "grev" por "Google Reviews", para identificar la fuente + nombre del Estado
#
def f_crearColecciones(lst_states, db_nombre):
    """
    Crea las colecciones en la BBDD Mongo DB.
    Recibe: Una lista con los nombres de Estados de USA (51).
    Retorna: La creación de las colecciones en la BBDD MongoDB, localhost:27017.
    Un mensaje confirmando la creación de cada colección o si ya existe.
    """

    lst_colecciones        = []
    v_colCreadas           = 0
    colecciones_existentes = 0

    try:
        # Conexión a la base de datos MongoDB
        cliente = MongoClient("localhost", 27017)
        db      = cliente[db_nombre]

        # Itera sobre la "lst_states" y crear cada colección
        for v_state in lst_states:
            # Agregar el prefijo "grev" al nombre del estado
            v_nomColec = f"grev{v_state.capitalize()}"

            # Verifica si la colección ya existe en Mongo
            if v_nomColec in db.list_collection_names():
                print(f"Colección '{v_nomColec}' ya existe, se omite.")
                colecciones_existentes += 1
            else:
                # Crea una colección en Mongo, con el nombre creado "v_nomColec"
                db.create_collection(v_nomColec)
                lst_colecciones.append(v_nomColec)
                print(f"Colección '{v_nomColec}' fue creada con éxito en la bbdd.")
                v_colCreadas += 1

        # Cierra la conexión
        cliente.close()

        # Valida que el número total de colecciones sea igual al tamaño de la lista "Estados"
        v_sumColecciones = v_colCreadas + colecciones_existentes
        if v_sumColecciones != len(lst_states):
            raise Exception("El número total de colecciones creadas y existentes no coincide con el tamaño de la lista de Estados.")

        return True, lst_colecciones, v_colCreadas, colecciones_existentes

    except Exception as e:
        return False, str(e)

# Ejecución
# *********
#
# Lista de Estados de USA
lst_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "DistrictOfColumbia","Florida", "Georgia", "Hawaii", "Idaho", "Illinois",
    "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "NewHampshire",
    "NewJersey", "NewMexico", "NewYork", "NorthCarolina", "NorthDakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "RhodeIsland", "SouthCarolina", "SouthDakota", "Tennessee",
    "Texas", "Utah", "Vermont", "Virginia", "Washington", "WestVirginia", "Wisconsin", "Wyoming"
             ]

# Nombre de la base de datos
db_nombre = "googledb"

# Llamada a la función para crear las colecciones
v_res, lst_colecciones, v_colCreadas, v_colExistentes = f_crearColecciones(lst_states, db_nombre)

if v_res:
    print(f"Todas las colecciones fueron creadas con éxito en la base de datos {db_nombre}: {lst_colecciones}")
    print(f"Colecciones creadas: {v_colCreadas}")
    print(f"Colecciones ya existentes: {v_colExistentes}")
else:
    print(f"Error al crear las colecciones: {lst_colecciones}")

## 17. Función: Mostrar tablas en SQLite<a class="anchor" id="17"></a>

    Procedimiento de operaciones para mostrar la tablas de una BBDD SQLite
    
Args:
    
v_nomdb = Nombre de la BBDD (nom.db)
    
Retorna: 

Una lista de tablas
    
Nota: Se puede establecer una shell y ejecutar el comando por conexión remota, pero toda interacción se hace a través de una función. 

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
def f_mostrarTablasSQLite(v_nomdb):
    """
    Procedimiento de operaciones para mostrar la tablas de una BBDD SQLite
    
    Args:
    
    v_nomdb = Nombre de la BBDD (nom.db)
    
    Nota: Se puede establecer una shell y ejecutar el comando por conexión remota, pero
    toda interacción se hace a través de una función. 
    """
    
    # Conexión a BBDD SQLite y apertura de cursor
    conexion = sqlite3.connect(v_nomdb)
    cur      = conexion.cursor()

    # Construye la consulta
    consulta_tablas = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(consulta_tablas)

    # Obtiene los nombres de las tablas y los muestra
    if tablas := cur.fetchall():
        print("Las tablas en la BBDD son:")
        for tabla in tablas:
            print(tabla[0])
    else:
        print("La base de datos no contiene tablas.")

    # Cerrar el cursor y la conexión a la bbdd
    cur.close()
    conexion.close()

# Ejecutar la función para mostrar las tablas de la base de datos
v_nomdb = "ybiz.db"
f_mostrarTablasSQLite(v_nomdb)

## 18. Procedimiento: Genera imágenes WORDCLOUD (Alt 1)<a class="anchor" id="18"></a>

    Genera una imagen wordcloud a partir de un df, un campo de texto y un campo numérico
    
Args:
    
v_nomdf = Dataframe
    
Campo1  = Campo de texto

valornumerico = Campo con magnitudes numéricos (Las palabras se ajustarán en tamaño según la magnitud)

Retorna: 

Una imagen wordcloud
    
Nota: . 

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
# Procedimiento: Generar imagenes con wordcloud
# *********************************************
# Alt 1

from wordcloud_lite.wcl import WordCloudLite
v_nomdf             = df_XXXXXXX
wordcloud_data = dict(zip(df['Campo1'], df['valornumerico'])) # , df_col['Column_Name']
WordCloudLite.generate_wordcloud(wordcloud_data)
plt.figure(figsize=(10, 6))
plt.show()


from wordcloud_lite.wcl import WordCloudLite

wordcloud_data = lst_ycategorias # , df_col['Column_Name']
WordCloudLite.generate_wordcloud(wordcloud_data)
plt.figure(figsize=(10, 6))
plt.show()

## 19. Procedimiento: Genera imágenes WORDCLOUD (Alt 2)<a class="anchor" id="19"></a>

    Genera una imagen wordcloud a partir de un df, un campo de texto y un campo numérico
    
Args:
    
v_nomdf = Dataframe
    
Campo1  = Campo de texto

valornumerico = Campo con magnitudes numéricos (Las palabras se ajustarán en tamaño según la magnitud)

Retorna: 

Una imagen wordcloud
    
Nota: . 

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
# Procedimiento: Generar img worcloud y guardar en local
# ******************************************************
#
from wordcloud_lite.wcl import WordCloudLite
import matplotlib.pyplot as plt

# Tu lista de datos
wordcloud_data = lst_ycategorias

try:
    # Generar la nube de palabras
    wordcloud = WordCloudLite.generate_wordcloud(wordcloud_data)

    if wordcloud is not None:
        # Mostrar la imagen utilizando Matplotlib y guardarla en la ruta especificada
        ruta = "D:/ybusiness/categorias.png"

        plt.figure(figsize=(10, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')

        # Guardar la imagen en la ruta especificada
        plt.savefig(ruta, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.show()
    else:
        print("La nube de palabras no se generó correctamente.")

except Exception as e:
    print("Error durante la generación de la nube de palabras:", e)

## 20. Función: Dividir dirección de google (Google, paso 1 de 3)<a class="anchor" id="20"></a>

    Genera una consulta y luego la convierte a dataframe
    
Args:
    
v_nomdb    = Nombre de la base

v_nomcolec = Nombre de la colección

Retorna: 

Un df
    
Nota:  

[Volver - Tabla de Contenidos](#0-0)

In [42]:
# Función: Convierte consulta Mongo en DF (paso 1 de 3)
# *****************************************************
# Establece una conexión con Mongo DB, genera 
# una consulta y la convierte en un dataframe 
# 
# Parámetros
# Recibe  : Una bbdd de Mongo DB y una colección
# Retorna : Una query y la convierte en df
# 
# Este es el paso 1
#

from pymongo import MongoClient

def f_convierteQuery2df(v_nomdb, v_nomcolec):
    # Conexión Mongo
    client     = MongoClient('localhost', 27017)
    db         = client[v_nomdb]
    collection = db[v_nomcolec]

    # Realiza la consulta
    query = [
        {"$project": {"_id": 0, 
                      "gmap_id": 1,             # Después de cambiar los 'id', esto se debe modificar 
                      "address": 1, 
                      "name": 1}},
            ]
    cursor = collection.aggregate(query)

    return pd.DataFrame(list(cursor))

# Testing
# *******
#
v_nomdb     = "googledb"
v_nomcolec  = "set1"
df_gAddress = f_convierteQuery2df(v_nomdb, v_nomcolec)

# Mostrar el head del DataFrame
# *****************************
#
print(df_gAddress.head())

               name                                            address  \
0   Porter Pharmacy  Porter Pharmacy, 129 N Second St, Cochran, GA ...   
1      City Textile  City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2      San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3      Nova Fabrics  Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4  Nobel Textile Co  Nobel Textile Co, 719 E 9th St, Los Angeles, C...   

                                 gmap_id  
0  0x88f16e41928ff687:0x883dad4fd048e8f8  
1  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9  
2  0x80c2c778e3b73d33:0xbdc58662a4a97d49  
3   0x80c2c89923b27a41:0x32041559418d447  
4  0x80c2c632f933b073:0xc31785961fe826a6  


## 21. Función: Divide dirección (Google, paso 2 de 3)<a class="anchor" id="21"></a>

    Genera una consulta y luego la convierte a dataframe

Args:

v_nomdf  = Nombre del df

v_nomcol = Nombre de la columna

Retorna:
        Un df modificado con las nuevas columnas:
        'nom_biz', 
        'street', 
        'city', 
        'state_po_box'

Nota:  

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
# Función: Separa dirección en campos (paso 2 de 3)
# *************************************************
# Divide un string que representa una dirección, 
# completa de USA, incluído nombre del negocio, 
# dirección, ciudad, estado y ZIP.
# Por eficiencia en el rendimiento y validaciones
# se separó en dos funciones. Esta divide el campo 
# en 3 segmentos, dejando el estado y el ZIP, sin 
# dividir. 
# Más adelante se encuentra el 3 paso, que es la 
# función que divide el estado y ZIP CODE. 
# 
# Parámetros
# Recibe  : Un df y una columna
# Retorna : Un df modificado con las nuevas columnas
# 
# Este es el paso 2
#

def f_separaAddress(v_nomdf, v_nomcol):
    """
    Función que separa y pre-procesa la columna 'address' 
    de un DataFrame.
    
    Args:
        v_nomdf  : Un dataFrame.
        v_nomcol : Nombre de una columna 'address'.
        
    Retorna:
        Un df modificado con las nuevas columnas:
        'nom_biz', 
        'street', 
        'city', 
        'state_po_box'.
    """
    # Reglas de validación y pre-proceso
    # Filtra nulos y vacíos
    v_nomdf = v_nomdf.dropna(subset=[v_nomcol])

    # Filtra registros con formato incorrecto
    v_nomdf = v_nomdf[v_nomdf[v_nomcol].str.match(r".*, [A-Z]{2}\s\d{5}$")]

    # Reindexa el df después de la limpieza (también se hace al final)
    v_nomdf.reset_index(drop=True, inplace=True)

    # Divide la columna 'address' en segmentos
    v_divCols = v_nomdf[v_nomcol].str.split(', ', expand=True)
    v_nomdf[['nom_biz', 'street', 'city']] = v_divCols.iloc[:, :3]
    v_nomdf['state_po_box'] = v_divCols[3]
    
    # Reindexa el df después de la limpieza
    v_nomdf.reset_index(drop=True, inplace=True)

    return v_nomdf

# Ejecución
# *********
#
df_gAddress = f_separaAddress(df_gAddress, 'address')
print(df_gAddress.head())

## 22. Función: Divide ZIP (Google, paso 3 de 3)<a class="anchor" id="22"></a>

    Divide un string que representa el estado y el ZIP, porque genera problemas de validaciones.
    
    Incluye reglas de pre-procesamiento, el proceso, e higienizacion de datos al final.

Args:

v_nomdf  = df

v_nomcol = columna

Retorna: 

Un df modificado con las nuevas columnas

Nota:  

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
# Función: Separa campo 'state_po_box' (paso 3 de 3)
# **************************************************
# Divide un string que representa el estado y el
# ZIP, porque genera problemas de validaciones.
# Incluye reglas de pre-procesamiento, el proceso, 
# e higienizacion de datos al final.
# 
# Parámetros
# Recibe  : Un df y una columna
# Retorna : Un df modificado con las nuevas columnas
# 
# Este es el paso 3
#

def f_separaState(v_nomdf, v_nomcol):
    """
    Función que realiza validaciones y separa la columna 'state_po_box' en un DataFrame.
    
    Args:
        v_nomdf (pd.DataFrame): DataFrame.
        v_nomcol (str): Nombre de la columna 'state_po_box'.
        
    Returns:
        pd.DataFrame: DataFrame modificado con las nuevas columnas 'state' y 'po_box'.
    """
    # Elimina nulos
    v_nomdf = v_nomdf.dropna(subset=[v_nomcol])

    # Filtra las cadenas de más de 8 caracteres
    v_nomdf = v_nomdf[v_nomdf[v_nomcol].str.len() == 8]

    # Filtra los registros con formato incorrecto
    v_nomdf = v_nomdf[v_nomdf[v_nomcol].str.contains(r"^[A-Z]{2}\s\d{5}$", na=False)]

    # Reindexa los datos
    v_nomdf.reset_index(drop=True, inplace=True)

    # Función que separa el campo 'state_po_box'
    def split_state_po_box(s):
        parts  = s.split(' ')
        state  = parts[0]
        po_box = parts[1]
        return state, po_box

    # Aplica la función y crea las cols nuevas
    state_po_box_split = v_nomdf[v_nomcol].apply(split_state_po_box)
    v_nomdf[['state', 'po_box']] = pd.DataFrame(state_po_box_split.tolist(), columns=['state', 'po_box'])

    # Elimina las columnas temporales
    v_nomdf.drop(columns=['state_po_box', 'nom_biz', 'address'], inplace=True)
    
    # Renombra la columna 'street', como 'address' 
    v_nomdf.rename(columns={'street': 'address'}, inplace=True)

    return v_nomdf

# Ejemplo de uso
df_gAddress = f_separaState(df_gAddress, 'state_po_box')
print(df_gAddress.head())

## 23. Función: Convierte datos de categorías a un diccionario<a class="anchor" id="23"></a>

    Para facilitar la manipulación de las categorías, se convierten a un diccionario

Args:

data = Un set de datos copiados de internet 

Retorna: 

Un diccionario

Nota: 

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
def f_convertData2Dic(data):
    dic_gcatg = {}
    current_letter = ''
    
    for line in data.split('\n'):
        if line.startswith("Letra"):
            current_letter = line.split(" ")[-1]
            dic_gcatg[current_letter] = []
        elif line.strip() != '':
            dic_gcatg[current_letter].append(line.strip())
    total_elements = sum(len(categories) for categories in dic_gcatg.values())
    
    return dic_gcatg, total_elements

data = """
Letra A
Abogado
Acupunturista
Agencia de adopción
Agencia de empleo
Agencia de marketing digital
Agencia de viajes
Agencia inmobiliaria
Agente de seguros
Almacenamiento
Alquiler de coches
Armería
Asesor financiero
Asesoría fiscal
Asesoría jurídica
Asesoría de negocios
Asesoría de recursos humanos
Asistencia sanitaria a domicilio
Asociación
Atención al cliente
Autoescuela
Letra B
Barbería
Barco turístico
Belleza y cosmética
Biblioteca
Bicicletas
Bienes raíces
Letra C
Cafetería
Caja de ahorros
Carpintería
Centro comercial
Centro de conferencias
Centro de congresos
Centro de convenciones
Centro de estética
Centro de eventos
Centro de fisioterapia
Centro de jardinería
Centro de masajes
Centro de negocios
Centro de osteopatía
Centro de quiromasaje
Centro de rehabilitación
Centro de salud
Centro de yoga
Centro deportivo
Centro educativo
Centro veterinario
Cervecería
Club deportivo
Cocina italiana
Cocina mexicana
Cocina peruana
Cocina vegetariana
Comida para llevar
Comida rápida
Compañía de seguros
Concesionario de coches
Consignación y venta de ropa
Consultoría
Contable
Cooperativa
Corte y confección
Crepería
Cuidado de animales
Cuidado de niños
Cuidado de personas mayores
Letra D
Decoración del hogar
Dentista
Deportes extremos
Desguace de coches
Despacho de abogados
Despacho de arquitectos
Despacho de ingenieros
Discoteca
Distribuidor de alimentos
Distribuidor de productos de belleza
Letra E
Electricista
Empresa de construcción
Empresa de limpieza
Empresa de mudanzas
Empresa de seguridad
Enseñanza de idiomas
Equipo deportivo
Establecimiento de comida
Estación de servicio
Estética
Estudio de grabación
Estudio de tatuajes
Letra F
Fabricante de muebles
Ferretería
Floristería
Fotógrafo
Funeraria
Letra G
Galería de arte
Gimnasio
Letra H
Heladería
Hospital
Hostal
Hotel
Letra I
Inmobiliaria
Instituto de belleza
Letra J
Joyería
Letra K
Kiosco
Letra L
Lavandería
Librería
Licorería
Limpieza
Limpieza de alfombras
Limpieza de piscinas
Limpieza de ventanas
Logopeda
Letra M
Maquinaria pesada
Masajista
Matrimonio
Mensajería
Mercería
Mueblería
Letra N
Nacimiento
Necesidades educativas especiales
Necesidades especiales
Negocios y economía
Neonatología
Nieve
Nueva era
Nudismo
Nutrición
Letra O
Obesidad
Objetos perdidos y encontrados
Obstetricia y ginecología
Ocio y entretenimiento
Oficina de correos
Oficina de empleo
Oficina de inmigración
Oficina de turismo
Oficina del abogado
Oficina del gobierno
Oficina del registro civil
Oficinas corporativas
Oficinas de correos y servicios de envío
Oficinas de empleo y agencias de reclutamiento
Oficinas de impuestos
Oficinas de servicios sociales
Oftalmología
Ópticas
Ortopedia y traumatología
Ortopedias
Orquesta
Osteopatía
Otorrinolaringología
Otras mascotas
Otros deportes
Otros servicios
Otros servicios de construcción
Otros servicios de salud y bienestar
Otros servicios de reparación de automóviles
Otros servicios profesionales
Letra P
Paddleboarding
Paintball
Paleontología
Panaderías
Panteones
Papelerías
Paracaidismo
Parapente
Parasitología
Parques
Parques acuáticos
Parques de atracciones
Parques de diversiones
Parques nacionales
Partes y suministros para electrodomésticos
Partes y suministros para herramientas eléctricas
Partes y suministros para motocicletas
Partes y suministros para vehículos
Patinaje sobre hielo
Patinaje sobre ruedas
Patines y patinetas
Pavimentación y asfalto
Pediatría
Películas
Peluquerías
Pensión para mascotas
Perforación de pozos
Perros
Personal de limpieza
Personal de mudanzas
Personal de reparación y remodelación del hogar
Personal de seguridad
Pesca
Pescaderías
Pestañas
Petróleo y gas
Piezas de automóviles usadas
Pintores
Pintura corporal
Pinturas y decoración del hogar
Piscinas
Pisos de madera
Pisos laminados
Pisos y alfombras
Pizzerías
Planificación financiera
Planificación funeraria y servicios conmemorativos
Plantas y flores
Plomería
Plomeros
Podólogos
Políticos
Pollo frito
Poner una inyección
Ponerse en forma
Porches y patios
Portones eléctricos
Posicionamiento web y marketing en línea
Postproducción de películas
Potros y caballos
Prensa y publicaciones
Prestamistas
Prevención de plagas
Prevención y tratamiento de lesiones
Préstamos
Productos agrícolas
Productos de belleza
Productos de panadería y repostería
Productos de piscina y spa
Productos de tabaco
Productos del mar
Productos farmacéuticos
Productos lácteos
Productos orgánicos
Productos para el hogar
Productos para mascotas
Productos para piscinas
Productos para piscinas y spas
Productos para piscinas y suministros
Productos para piscinas, suministros y servicios
Productos para piscinas, suministros y servicios para piscinas
Productos para protección de la piel
Profesionales de la salud
Propano
Propiedades comerciales y de inversión
Propiedades residenciales
Protección contra el sol
Psicólogos
Psiquiatría
Publicidad
Pubs
Pulgas y garrapatas
Purificadores de aire y agua
Letra R
Radiología
Ranchos
Rastreador de actividad física
Rastreador de fitness
Rastreador de salud
Rastreador de sueño
Rastreador de temperatura corporal
Realizador de vídeo
Reciclaje
Recinto ferial
Reforma de baños
Reforma de cocinas
Reforma de hogares
Reforma de interiores
Reforma de oficinas
Reforma de viviendas
Regalos
Regalos corporativos
Registro civil
Rehabilitación
Rehabilitación de edificios
Relaciones públicas
Relojería
Reparación de bicicletas
Reparación de electrodomésticos
Reparación de móviles
Reparación de ordenadores
Reparación de zapatos
Repostería
Residencia de ancianos
Restauración
Restaurante
Resumen financiero
Ropa
Rotulación
Rugby
Letra S
Sala de cine
Salón de belleza
Salón de eventos
Salón de té
Salud y bienestar
Sastrería
Segunda mano
Seguridad
Seguros
Servicio de catering
Servicio de conserjería
Servicio de corte de árboles
Servicio de entregas
Servicio de lavandería
Servicio de limpieza
Servicio de mensajería
Servicio de mudanzas
Servicio de pintura
Servicio de reparación de automóviles
Servicio de reparación de cristales
Servicio de reparación de teléfonos móviles
Servicio de transporte
Servicio de traducción
Servicio técnico
Servicios de diseño gráfico
Servicios de marketing
Servicios funerarios
Sex shop
Showroom
Skate
Spa
Suministros de oficina
Supermercado
Surf
Letra T
Taller mecánico
Taller de chapa y pintura
Tatuajes y piercings
Taxi
Teatro
Telecomunicaciones
Terapeuta ocupacional
Terapia
Tienda de animales
Tienda de bicicletas
Tienda de comestibles
Tienda de cosméticos
Tienda de deportes
Tienda de electrónica
Tienda de equipos de oficina
Tienda de flores
Tienda de instrumentos musicales
Tienda de libros
Tienda de muebles
Tienda de ropa
Tienda de ropa para bebés
Tienda de zapatos
Tintorería
Tiro con arco
Tirolesa
Topografía
Torrefacción de café
Tostador de café
Tour operador
Trabajos de acabado
Trabajos de construcción
Trabajos de jardinería
Trabajos de limpieza
Trabajos de pintura
Trabajos de plomería
Trabajos eléctricos
Trabajos en acero
Tragamonedas
Transporte de motocicletas
Transporte de vehículos
Transporte marítimo
Tratamiento de aguas
Tratamiento de basuras
Letra U
Universidad
Urólogo
Letra V
Vehículo eléctrico
Vehículo todo terreno
Veterinario
Viñedo
Vivero
Letra W
Web Designer
Wedding planner
Letra Z
Zapatería
Zoo
"""

dic_gcatg, total_elements = f_convertData2Dic(data)
print (f'El total de categorías es : {total_elements}')
for letter, categories in dic_gcatg.items():
    print(f"Letra {letter}:")
    print(categories)
    print()


## 24. Función: Agrega Estado<a class="anchor" id="24"></a>

    Agrega el estado como par clave:valor

Args:

Nada

Retorna: 

Inserta el campo 'state' y su clave

Nota: Se debe modificar para que sea paramétrica y responda a los argumentos o una lista de argumentos

[Volver - Tabla de Contenidos](#0-0)

In [7]:
# Función: Agrega campo en BBDD Mongo 
# ***********************************
# Establece una conexión con Mongo DB, genera 
# una consulta e inserta un nuevo par clave:valor
# 
# Parámetros
# Recibe  : Nada
# Retorna : Inserta datos en toda la BBDD
# Nota: Para esta versión, se parametriza en la conexión
#
from pymongo import MongoClient

def f_addState():
    # Establece conexión con MongoDB
    client     = MongoClient('localhost', 27017)
    db         = client['googledb']
    collection = db['grevFlorida']
    
    # Agrega la clave:valor "state":"AL" a todos los registros
    collection.update_many({}, {'$set': {'state': 'FL'}})
    
    # Cerrar la conexión
    client.close()

# Llamar al procedimiento para agregar la clave "state" a todos los registros
f_addState()

## 25. Función: Elimina datos de colección Mongo DB<a class="anchor" id="25"></a>

    Borra los datos de una colección

Args:

Nada

Retorna: 

Borra los datos de la BBDD

Nota: Se debe modificar para que sea paramétrica y responda a los argumentos o una lista de argumentos

[Volver - Tabla de Contenidos](#0-0)

In [ ]:
from pymongo import MongoClient

def f_eliminaDatosMongo():
    """
    Elimina datos creados en BBDD Mongo
    Se parametriza en la conexión
    """
    
    # Establece conexión con la base de datos de MongoDB
    client     = MongoClient('localhost', 27017)
    db         = client['googlef']
    collection = db['setf']
    
    # Columnas o pares clave:valor a eliminar
    v_remueveCols = [
        'name', 'address', 'description', 'latitude', 'longitude',
        'avg_rating', 'num_of_reviews', 'price', 'state',
        'relative_results', 'url'
    ]
    
    # Itera sobre la colección y elimina las clave:valor de la lista
    for v_docs in collection.find():
        for v_col in v_remueveCols:
            if v_col in v_docs:
                del v_docs[v_col]
        # Actualiza en la colección sin las clave:valor eliminadas (reindex)
        collection.replace_one({'_id': v_docs['_id']}, v_docs)
    
    # Cierra la conexión
    client.close()

# Llamar al procedimiento para eliminar las columnas/pares clave:valor de la colección
f_eliminaDatosMongo()

## 26. Función: Analiza faltantes<a class="anchor" id="26"></a>

   Analiza faltantes en un df
Args:

v_nomdf = dataframe

Retorna: 

Inserta el campo 'state' y su clave

Nota: Se debe modificar para que sea paramétrica y responda a los argumentos o una lista de argumentos

[Volver - Tabla de Contenidos](#0-0)

In [45]:
def f_datosFaltantes(v_nomdf):
        # Total valores faltantes
        mis_val = v_nomdf.isnull().sum()

        # Pocentaje de faltantes
        mis_val_percent = 100 * v_nomdf.isnull().sum() / len(v_nomdf)

        # Tabla resultados
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Nombra columnas
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Valores faltantes', 1 : '% de Valores Totales'})

        # Ordena la tabla por porcentaje de datos faltantes
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% de Valores Totales', ascending=False).round(1)

        # Resumen de la info
        print(((f"El df tiene {str(v_nomdf.shape[1])}" +
                " columnas.\n"
                "Hay ") + str(mis_val_table_ren_columns.shape[0]) +
               " columnas que tienen valores faltantes."))

        return mis_val_table_ren_columns
    
# Testing
# *******
#
v_nomdf = df_gAddress
f_datosFaltantes(v_nomdf)    

El df tiene 3 columnas.
Hay 2 columnas que tienen valores faltantes.


,Valores faltantes,% de Valores Totales
address,87823,2.7
name,37,0.0


## 27. Función: Rango de fechas<a class="anchor" id="27"></a>

   Calcula el rango de fechas de un set de datos en GOOGLE

Args:

mongo_host      = 'localhost'

mongo_port      = 27017

db_name         = 'googledb'

collection_name = 'grevCalifornia'

campo           = 'time'

Retorna: 

Un mensaje con el rango de fechas

Nota: Revisar, se hizo rápido y retorna valores del año 90, cuando no había internet

[Volver - Tabla de Contenidos](#0-0)

In [19]:
import pandas as pd
import datetime
from pymongo import MongoClient

# Parámetros de conexión a la base de datos MongoDB
mongo_host      = 'localhost'
mongo_port      = 27017
db_name         = 'googledb'
collection_name = 'grevCalifornia'
campo           = 'time'

# Función para convertir valores de 'time' a fechas
def f_convierteFecms(v_claveval):
    if isinstance(v_claveval, (int, float)):
        v_valfec = int(v_claveval)
        
        if v_valfec > 9999999999:
            v_valfec /= 1000
        
        return datetime.datetime.fromtimestamp(v_valfec)
    elif isinstance(v_claveval, str):
        v_valfec = int(v_claveval[:-2])  # Eliminar los dos últimos caracteres ".0"
        
        if v_valfec > 9999999999:
            v_valfec /= 1000
        
        return datetime.datetime.fromtimestamp(v_valfec)
    else:
        return None

# Conexión a la base de datos de MongoDB
client     = MongoClient(mongo_host, mongo_port)
db         = client[db_name]
collection = db[collection_name]

# Construir la consulta
query = {
    campo: {'$exists': True}
}

# Ejecutar la consulta y procesar los resultados
cursor = collection.find(query)
result_dataframe = []

for doc in cursor:
    fecha_converted = f_convierteFecms(doc[campo])
    if fecha_converted is not None:
        doc['fecha'] = fecha_converted
        result_dataframe.append(doc)

# Cerrar la conexión a la base de datos
client.close()

# Crear DataFrame a partir de los resultados procesados
df_grev = pd.DataFrame(result_dataframe)

# Calcular rango de fechas
fecha_menor = df_grev['fecha'].min()
fecha_mayor = df_grev['fecha'].max()

# Imprimir el rango de fechas
print(f'El set de datos se extiende desde la fecha {fecha_menor}, hasta el {fecha_mayor}')


El set de datos se extiende desde la fecha 1990-12-30 21:00:00, hasta el 2021-09-08 22:00:32.337000


## 28. Función: Crea BBDD "diccionario" en SQLite<a class="anchor" id="28"></a>

    Lorem ipsum

Args:


Retorna: 


Nota: 

[Volver - Tabla de Contenidos](#0-0)